# CH06. Advanced URL and Name space
---
이번 장에서는 템플릿에서 사용된 URL의 하드코딩을 없애는 방법에 대해서 정리하자.<br>
### 1. URL 하드코딩
먼저 quetion_list.html 템플릿에 사용된 링크는 다음과 같다.<br>
```html
<li><a = href="/myapp/{{ question.id }}">{{ question.subject }}</a></li>
```
상세조회를 위한 URL 링크이다. 하지만 이러한 URL 규칙은 웹 프로그램이 확장될 때 마다 수정될 가능성이 매우 크다. 예를 들어<br>
"http://localhost:8000/myapp/question/2" 또는<br>
"http://localhost:8000/myapp/2/question" 등으로 바뀔수 있기 때문이다.<br><br>
URL 규칙이 자주 변경된다면 템플릿들에서 사용된 모든 URL들을 일일이 찾아가며 수정해 주어야하는 리스크가 발생하게 된다. 이러한 문제점을 해결하기 위해서는 해당 URL에 대한 실제 링크대신 링크의 주소가 매핑되어 있는 별칭을 사용해야 한다.

### 2. URL 별칭
링크의 주소대신 별칭으로 사용하려면 URL 매핑에 name 속성을 부여해 주어야 한다.<br>
``` myapp/urls.py ```
```python
from django.urls import path

from . import views

urlpatterns = [
# ----------------------------[ edit ]------------------------------ #
    path('', views.index, name='index'),
    path('<int:question_id>/',views.detail, name='detail'),
# ------------------------------------------------------------------ #
]
```
이렇게 http://localhost:8000/myapp/은 index,<br>
http://localhost:8000/myapp/2 는 detail 이라는 이름을 부여했다.<br>

### 3. 템플릿에서 URL 별칭 사용하기
이렇게 myapp/urls.py 파일 위와 같이 변경하면 템플릿에서 다음처럼 사용할 수 있다.<br>
``` templates/myapp/question_list.html ```
```html
...
    {% for question in question_list %}
<!-- -----------------[ edit ]-------------------- -->
        <li><a href="{% url 'detail' question.id %}"></a>{{ question.subject }}</li>
<!-- --------------------------------------------- -->
    {% endfor %}
...
```
하드 코딩 되어 있던 ``` /myapp/{{ question.id }} ``` 링크가 ``` {% url 'detail' question.id %} ```로 변경되었다. 여기서 ``` question.id ```는 URL 매핑에서 정의된 ``` <int:question_id> ```를 의미한다.<br><br>

### 4. URL 네임스페이스
한가지 더 생각해 볼 문제가 있다. 현재는 myapp 앱 하나만 작성중이지만 myapp앱 이외의 다른 앱이 프로젝트에 추가 될 수도 있을 것이다. 이럴 경우 서로 다른 앱에서 동일한 URL 별칭을 사용한다면 중복이 발생하게 될 것이다.<br>
이 문제를 해결하려면 myapp/urls.py 파일에 네임스페이스에 해당되는 app_name 변수를 지정해야한다. <br>
다음처럼 myapp/urls.py 파일에 app_name을 설정해 보자.<br>
``` myapp/urls.py ```
```python
...
app_name = 'myapp'
...
```
이후에 템플릿에서 사용된 URL에 네임스페이스를 지정해 주어야 한다.
``` templates/myapp/question_list.html ```
```html
...
    {% for question in question_list %}
<!-- ----------------[ edit ]------------------- -->
        <li><a href="{% url 'myapp:detail' question.id %}">{{ question.subject }}</a></li>
<!-- ------------------------------------------- -->
    {% endfor %}
...
```
<br>
``` {% url 'detail' question.id %} ```이라는 URL 별칭을 ``` {% url 'myapp:detail' question.id %} ```로 바꾸어 주었다.<br><br>

그냥 detail 앞에 myapp이라는 네임스페이스를 붙여준 것이다.